<a href="https://colab.research.google.com/github/gdmcdonald/Cutting-Stock-Problem/blob/main/solve_cutting_stock_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve cutting problem

Solves a problem in which various lengths of wood must be cut with minimal waste from longer lengths of wood. https://en.wikipedia.org/wiki/Cutting_stock_problem

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [ ]:
!pip install ortools


Solves a problem in which various lengths of wood must be cut with minimal waste from longer lengths of wood. https://en.wikipedia.org/wiki/Cutting_stock_problem


In [13]:
from ortools.linear_solver import pywraplp


def create_data_model():
    """Create the data for the example."""
    data = {}
    # These are the lengths you want to cut
    lengths = [910, 837.8, 773.2, 716.2, 666.8, 625, 590.8, 564.2, 545.2, 533.8, 530, 533.8, 545.2, 564.2, 590.8, 625, 666.8, 716.2, 773.2, 837.8, 910]
    data['lengths'] = lengths
    data['items'] = list(range(len(lengths)))
    # These are the stock you have available
    data['stock_lengths'] = [2400, 2400, 2400, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800, 1800]
    data['cut_width'] = 3 #how many mm the saw takes out of the stock.
    data['stock'] = list(range(len(data['stock_lengths'])))
    return data



def main():
    data = create_data_model()

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        return

    # Variables
    # x[i, j] = 1 if item i is packed in stock j.
    x = {}
    for i in data['items']:
        for j in data['stock']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

    # y[j] = 1 if stock j is used.
    y = {}
    for j in data['stock']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

    # Constraints
    # Each item must be in exactly one stock.
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['stock']) == 1)

    # The amount packed in each stock_length cannot exceed its capacity.
    for j in data['stock']:
        solver.Add(
            sum(x[(i, j)] * (data['lengths'][i] + data['cut_width']) for i in data['items']) - data['cut_width'] <= y[j] *
            data['stock_lengths'][j])

    # Objective: minimize the number of stock used.
    solver.Minimize(solver.Sum([y[j] for j in data['stock']]))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        num_stock = 0
        for j in data['stock']:
            if y[j].solution_value() == 1:
                stock_items = []
                stock_length_used = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        stock_items.append(i)
                        stock_length_used += data['lengths'][i] 
                stock_length_used += (len(stock_items) - 1) * data['cut_width']
                if stock_items:
                    num_stock += 1
                    print('Stock number', j)
                    print('  Items packed:', stock_items)
                    print('  Lengths cut:', [data["lengths"][k] for k in stock_items])
                    print(f'  Total length used: {stock_length_used} out of {data["stock_lengths"][j]}')
                    print()
        print()
        print('Number of stock used:', num_stock)
        print('Time = ', solver.WallTime(), ' milliseconds')
    else:
        print('The problem does not have an optimal solution.')


if __name__ == '__main__':
    main()

Stock number 0
  Items packed: [1, 2, 18]
  Lengths cut: [837.8, 773.2, 773.2]
  Total length used: 2390.2 out of 2400

Stock number 1
  Items packed: [0, 15, 19]
  Lengths cut: [910, 625, 837.8]
  Total length used: 2378.8 out of 2400

Stock number 2
  Items packed: [5, 8, 13, 14]
  Lengths cut: [625, 545.2, 564.2, 590.8]
  Total length used: 2334.2 out of 2400

Stock number 6
  Items packed: [17, 20]
  Lengths cut: [716.2, 910]
  Total length used: 1629.2 out of 1800

Stock number 7
  Items packed: [7, 12, 16]
  Lengths cut: [564.2, 545.2, 666.8]
  Total length used: 1782.2 out of 1800

Stock number 10
  Items packed: [3, 9, 10]
  Lengths cut: [716.2, 533.8, 530]
  Total length used: 1786.0 out of 1800

Stock number 11
  Items packed: [4, 6, 11]
  Lengths cut: [666.8, 590.8, 533.8]
  Total length used: 1797.3999999999999 out of 1800


Number of stock used: 7
Time =  5475  milliseconds
